# Read the main data file

In [18]:
library("tidyverse")

In [19]:
df = read_csv(file = "../../data/processed/wnv.trap.date.rev3b.csv")

Parsed with column specification:
cols(
  .default = col_double(),
  t_date = col_date(format = ""),
  t_yr = col_integer(),
  t_mo = col_integer(),
  t_day = col_integer(),
  t_qtr = col_integer(),
  t_wk = col_integer(),
  t_day_of_yr = col_integer(),
  t_day_of_wk = col_integer(),
  t_day_of_wk_name = col_character(),
  t_eval_day = col_integer(),
  t_eval_wk = col_integer(),
  part_train = col_logical(),
  part_validate = col_logical(),
  part_test = col_logical(),
  part_partition = col_character(),
  trap_trap_name = col_character(),
  loc_lat_lng_src = col_character(),
  trap_satellite_ind = col_logical(),
  loc_zipcode = col_integer(),
  loc_community = col_character()
  # ... with 33 more columns
)
See spec(...) for full column specifications.


In [20]:
glimpse(df)

Observations: 13,631
Variables: 117
$ t_date                             <date> 2007-05-29, 2007-05-29, 2007-05...
$ t_yr                               <int> 2007, 2007, 2007, 2007, 2007, 20...
$ t_mo                               <int> 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,...
$ t_day                              <int> 29, 29, 29, 29, 29, 29, 29, 29, ...
$ t_qtr                              <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...
$ t_wk                               <int> 22, 22, 22, 22, 22, 22, 22, 22, ...
$ t_day_of_yr                        <int> 149, 149, 149, 149, 149, 149, 14...
$ t_day_of_wk                        <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...
$ t_day_of_wk_name                   <chr> "Mon", "Mon", "Mon", "Mon", "Mon...
$ t_eval_day                         <int> 514, 514, 514, 514, 514, 514, 51...
$ t_eval_wk                          <int> 74, 74, 74, 74, 74, 74, 74, 74, ...
$ part_train                         <lgl> TRUE, TRUE, TRUE, TRUE, TRUE, TR...
$ part_validate 

In [21]:
goog_data = read_csv("../../data//processed//google_trends.csv") %>%
    rename(ym = date)

Parsed with column specification:
cols(
  date = col_character(),
  googtrend_sym_wnv = col_integer(),
  googtrend_deadbirds = col_integer(),
  googtrend_westnile = col_integer(),
  googtrend_mosq_bites = col_integer()
)


In [22]:
glimpse(goog_data)

Observations: 181
Variables: 5
$ ym                   <chr> "2004-01", "2004-02", "2004-03", "2004-04", "2...
$ googtrend_sym_wnv    <int> 0, 36, 0, 0, 0, 96, 32, 30, 57, 0, 0, 0, 0, 0,...
$ googtrend_deadbirds  <int> 0, 0, 5, 0, 0, 0, 4, 3, 0, 0, 0, 3, 0, 0, 0, 0...
$ googtrend_westnile   <int> 5, 6, 8, 14, 20, 91, 75, 44, 37, 16, 0, 5, 6, ...
$ googtrend_mosq_bites <int> 0, 0, 0, 0, 0, 26, 15, 7, 20, 0, 0, 0, 0, 0, 5...


In [23]:
df = df %>% mutate(ym = paste0(t_yr,"-",
                               stringr::str_pad(t_mo,side = "left",width = 2,pad = "0"))) %>%
    left_join(goog_data) %>%
    select(-ym)

Joining, by = "ym"


# Data Type Conversions

In [24]:
factor_vars = c("loc_zipcode", "loc_census_block_group_id","loc_census_tract_id","zone_type")
for (col in factor_vars){
    df[,col] = as.factor(df[,col][[1]])
}

In [25]:
library("forcats")

In [26]:
df$loc_zipcode = fct_infreq(df$loc_zipcode)
df$loc_census_block_group_id = fct_infreq(df$loc_census_block_group_id)
df$loc_census_tract_id = fct_infreq(df$loc_census_tract_id)
df$zone_type = fct_infreq(df$zone_type)

### Add Mosq Species

In [33]:
add_active_species = function(df){
    df_1 = df %>% select(t_mo,starts_with("mos")) %>% rownames_to_column(var = "rowid") %>%
            select(-contains("wnv"),-contains("tot")) %>%
            gather(key,value,-rowid,-t_mo,mos_erraticus_num_mosquitos:mos_unspecified_num_mosquitos) %>%
            mutate(keeper = value>0,
                mos_species = stringr::str_extract(key,pattern = "(?![mos_])[\\w]*(?=_num)")) %>%
            filter(keeper)
    df %>% rownames_to_column(var = "rowid") %>% left_join(df_1[,c("rowid","mos_species")]) %>% select(-rowid)
}

In [34]:
df = add_active_species(df)

Joining, by = "rowid"


### Make all character vars into factor vars

In [36]:
char_cols = df %>% select_if(is_character) %>% names
df[char_cols] = map_df(df[char_cols],~as.factor(.x))

# Split into train, validation and test datasets

In [38]:
df %>% count(part_partition)

part_partition,n
test,3058
train,15257
validate,5991


In [39]:
partition_vars = df %>% select(starts_with("part_")) %>% names()

In [40]:
df_sub = df %>% select(-partition_vars)

In [41]:
df_train = df_sub[df$part_partition=="train",]
df_val   = df_sub[df$part_partition=="validate",]
df_test  = df_sub[df$part_partition=="test",]

What are the date ranges for each of these splits?

In [42]:
range(df_train$t_date)

[1] "2007-05-29" "2013-09-26"

In [43]:
range(df_val$t_date)

[1] "2014-06-05" "2016-09-29"

In [44]:
range(df_test$t_date)

[1] "2017-06-16" "2018-09-27"

# Save to file

In [45]:
saveRDS(df_train,file = "df_train.RData")
saveRDS(df_val,file = "df_val.RData")
saveRDS(df_test,file = "df_test.RData")